In [1]:
%load_ext autoreload
%autoreload 2

import os

os.chdir('..') # run once

In [12]:
from src.retrieval_model import fully_connected, embedding_model

In [5]:
embedding_model

'Builds a two branch network embedding model\n\n    Args:\n        img_feats (tuple): Shape of image features\n        sent_feats (tuple): Shape of sentence features\n\n    Returns:\n        Model: Embedding model\n    '